## Tests of Repliseq Track

Recapitulate Y Scaled (Z-Score) and treat as one-way ANOVA table

In [4]:
source("_targets.R")
library(dplyr)
library(GenomicRanges)
library(tibble)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:dplyr’:

    first, rename


The following object is masked from ‘package:utils’:

    findMatches


The following objects are mask

In [6]:
tar_load(
  c(
    chromosome_pericetromere_label,
    repli.timing_Germline_chr,
    repli.timing_Somatic_chr,
    repli.timing_Kc167_chr,
    repli.timing_S2_chr
  )
)

In [24]:
Y <- tibble(
  chr = c(names(chr.lengths), "rDNA"),
  xmin = rep(2500, length(chr)) %>% replace(match("rDNA", chr), 37500),
  xmax = floor((c(chr.lengths, seqlengths(repli.timing_Germline_chr)["rDNA"]) - 2500) / 5000) * 5000,
  Germline = list(repli.timing_Germline_chr),
  Somatic = list(repli.timing_Somatic_chr),
  Kc167 = list(repli.timing_Kc167_chr),
  S2 = list(repli.timing_S2_chr),
) %>%
  group_by(chr, xmin, xmax) %>%
  reframe(
    x = seq(xmin, xmax, by = 5000),
    region = (
      as.character(chr) %>%
        paste0(
          replace(
            rep("", length(x)),
            chromosome_pericetromere_label %>%
              findOverlaps(
                GRanges(
                  chr,
                  IRanges(x, width=1),
                  seqlengths = seqlengths(repli.timing_Germline_chr)
                )
              ) %>%
              to(),
            "C"
          )
        ) %>%
      factor(
        c("2L", "2LC", "2RC", "2R", "3L", "3LC", "3RC", "3R", "4", "X", "Y", "rDNA")
      )
    ),
    across(
      !matches("^(x|region)$"),
      list(
        LOESS = ~ (
          loess(
            y.track ~ x.track,
            tibble(
              x.track = .x[[1]][seqnames(.x[[1]]) == chr] %>% mid(),
              y.track = .x[[1]][seqnames(.x[[1]]) == chr]$score,
            ),
            span = if (chr == "rDNA") {
              0.1
            } else {
              0.025
            }
          ) %>%
          predict(tibble(x.track = x))
        )
      )
    )
  ) %>%
  subset(select = -c(xmin, xmax))
subset(Y, region == "rDNA")

chr,x,region,Germline_LOESS,Somatic_LOESS,Kc167_LOESS,S2_LOESS
<chr>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
rDNA,37500,rDNA,0.6482017,-0.6204086,-0.9240146,-0.8223618
rDNA,42500,rDNA,0.7099135,-0.5879558,-0.9338242,-0.9026897
rDNA,47500,rDNA,0.7568527,-0.6177137,-0.9291707,-0.8665072
rDNA,52500,rDNA,-0.4198505,-0.6580948,-0.9769341,-0.9492365
rDNA,57500,rDNA,0.6880967,-0.6337990,-0.8950174,-0.8244144
rDNA,62500,rDNA,0.6249556,-0.3777080,-0.8714755,-0.8432189
rDNA,67500,rDNA,0.7103718,-0.5854864,-0.9253684,-0.8596512


In [32]:
sapply(
  levels(Y$region),
  \(n) with(
    subset(Y, region == n & x %% 20000 == 2500),
    t.test(qlogistanh(Germline_LOESS), qlogistanh(Somatic_LOESS))
  ),
  simplify=FALSE
)

$`2L`

	Welch Two Sample t-test

data:  qlogistanh(Germline_LOESS) and qlogistanh(Somatic_LOESS)
t = 6.5089, df = 2213.6, p-value = 9.333e-11
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.1578006 0.2938865
sample estimates:
   mean of x    mean of y 
 0.224035555 -0.001807994 


$`2LC`

	Welch Two Sample t-test

data:  qlogistanh(Germline_LOESS) and qlogistanh(Somatic_LOESS)
t = 11.229, df = 94.68, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.8454739 1.2086450
sample estimates:
 mean of x  mean of y 
-0.1369713 -1.1640307 


$`2RC`

	Welch Two Sample t-test

data:  qlogistanh(Germline_LOESS) and qlogistanh(Somatic_LOESS)
t = 5.5222, df = 421.81, p-value = 5.864e-08
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.2139929 0.4505283
sample estimates:
 mean of x  mean of y 
-0.6962408 -1.0285014 


$`2R`

	W

In [25]:
YS <- Y[
  c("Germline_LOESS", "Somatic_LOESS", "Kc167_LOESS", "S2_LOESS")
] %>%
  as.matrix() %>%
  scale() %>%
  t() %>%
  scale() %>%
  t()
YS[Y$chr == "Y", ] <- (
  YS[Y$chr == "Y", c(1, 2, 4)] %>%
    t() %>%
    scale() %>%
    t() %>%
    `%*%`(
      matrix(
        c(
          1, 0, 0,
          0, 1, 0,
          0, 0, 0,
          0, 0, 1
        ),
        nrow = 3,
        dimnames = list(
          NULL,
          colnames(Y)[4:7]
        )
      )
    )
)
tail(YS)

,Germline_LOESS,Somatic_LOESS,Kc167_LOESS,S2_LOESS
"[27504,]",1.4416166,-0.196228455,-0.39059482,-0.8547933
"[27505,]",1.4583548,-0.252279529,-0.40024781,-0.8058275
"[27506,]",0.9181662,0.451990640,0.02696463,-1.3971214
"[27507,]",1.4642864,-0.287717381,-0.38712951,-0.7894395
"[27508,]",1.3859842,0.000566436,-0.45059642,-0.9359542
"[27509,]",1.4510692,-0.217053878,-0.40977718,-0.8242381


In [21]:
table(Y$region)


  2L  2LC  2RC   2R   3L  3LC  3RC   3R    4    X    Y rDNA 
4438  264 1130 3926 4631  990  846 5569  269 4707  732    7 

In [ ]:
sapply(
  levels(Y$region),
  \(n) YS %>%
    subset(Y$region == n & Y$x %% 20000 == 2500) %>%
    melt() %>%
    lm(value ~ Var2, .) %>%
    anova(),
  simplify=FALSE
)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
Var2,3,7.519516,2.5065052,3.346553,0.01831749
Residuals,4436,3322.480484,0.7489812,NA,NA
,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
Var2,3,128.3176,42.7725318,159.5935,1.14003e-58
Residuals,260,69.6824,0.2680092,NA,NA
,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
Var2,3,80.93249,26.9774969,39.61972,2.414984e-24


Fisher's Exact Test of Repli Peaks

In [4]:
source("_targets.R")
library(dplyr)
library(GenomicRanges)
library(withr)

Loading required package: viridisLite


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading r

In [6]:
tar_load(
  c(
    chromosome_pericetromere_label,
    repli.peaks_chr
  )
)

In [7]:
tabulate_germline_earlier <- substitute(
  . %>%
      subset(grepl("GermlineEarlier", names(.))) %>%
      `seqlevels<-`(value = c(seqlevels(.), "2LC", "2RC", "3LC", "3RC")) %>%
      `$<-`(
        "region",
        value = paste0(
          as.character(seqnames(.)),
          replace(
            rep(
              "",
              length(.)
            ),
            findOverlaps(
              chromosome_pericetromere_label,
              .
            ) %>%
              to(),
            "C"
          )
        ) %>%
          factor(levels = region)
      ) %>%
      elementMetadata() %>%
      as.data.frame() %>%
      pull("region") %>%
      tabulate()
)

In [8]:
sapply(
  repli.peaks_chr,
  \(gr) str_extract(names(gr), "\\.([A-z]+)", group=1) %>% table()
)

$Germline_Somatic
.
GermlineEarlier   GermlineLater 
            332             203 

$Germline_Kc167
.
GermlineEarlier   GermlineLater 
            467             273 

$Germline_S2
.
GermlineEarlier   GermlineLater 
            402             309 

$Somatic_Kc167
.
SomaticEarlier   SomaticLater 
           299            302 

$Somatic_S2
.
SomaticEarlier   SomaticLater 
           264            308 

$Kc167_S2
.
 Kc 
766 

$Dynamic_Static_Model
.
GermlineEarlier   GermlineLater 
            245             263 


In [9]:
regions <- tibble(
  region = c(
    "2L", "2LC", "2RC", "2R",
    "3L", "3LC", "3RC", "3R",
    "4", "X", "Y"
  ),
  Germline_Somatic = list(
    list(
      repli.peaks_chr$Germline_Somatic %>%
        subset(grepl("GermlineEarlier", names(.))),
      repli.peaks_chr$Germline_Somatic %>%
        subset(grepl("GermlineLater", names(.)))
    )
  ),
  Germline_Kc167 = list(
    list(
      repli.peaks_chr$Germline_Kc167 %>%
        subset(grepl("GermlineEarlier", names(.))),
      repli.peaks_chr$Germline_Kc167 %>%
        subset(grepl("GermlineLater", names(.)))
    )
  ),
  Germline_S2 = list(
    list(
      repli.peaks_chr$Germline_S2 %>%
        subset(grepl("GermlineEarlier", names(.))),
      repli.peaks_chr$Germline_S2 %>%
        subset(grepl("GermlineLater", names(.)))
    )
  ),
  Somatic_Kc167 = list(
    list(
      repli.peaks_chr$Somatic_Kc167 %>%
        subset(grepl("SomaticEarlier", names(.))),
      repli.peaks_chr$Somatic_Kc167 %>%
        subset(grepl("SomaticLater", names(.)))
    )
  ),
  Somatic_S2 = list(
    list(
      repli.peaks_chr$Somatic_S2 %>%
        subset(grepl("SomaticEarlier", names(.))),
      repli.peaks_chr$Somatic_S2 %>%
        subset(grepl("SomaticLater", names(.)))
    )
  ),
  Kc167_S2 = list(
    list(
      repli.peaks_chr$Kc167_S2 %>%
        subset(grepl("Kc167Earlier", names(.))),
      repli.peaks_chr$Kc167_S2 %>%
        subset(grepl("Kc167Later", names(.)))
    )
  ),
) %>%
  reframe(
    region,
    across(
      !region,
      \(gr_copy) sapply(
        gr_copy[[1]],
        \(gr) gr %>%
          `seqlevels<-`(value = c(seqlevels(.), "2LC", "2RC", "3LC", "3RC")) %>%
          `$<-`(
            "region",
            value = paste0(
              as.character(seqnames(.)),
              replace(
                rep(
                  "",
                  length(.)
                ),
                findOverlaps(
                  chromosome_pericetromere_label,
                  .
                ) %>%
                  to(),
                "C"
              )
            ) %>%
              factor(levels = region)
          ) %>%
          elementMetadata() %>%
          as.data.frame() %>%
          pull("region") %>%
          tabulate()
      )
    )
  ) %>%
  print(width=500)

# A tibble: 11 × 7
   region Germline_Somatic[,1]  [,2] Germline_Kc167[,1]  [,2] Germline_S2[,1]
   <chr>                 <int> <int>              <int> <int>           <int>
 1 2L                       59    42                 65    40              68
 2 2LC                       8     0                  7     4               6
 3 2RC                      20     1                 28    14              16
 4 2R                       42    31                 50    50              57
 5 3L                       54    43                 70    50              72
 6 3LC                      12     0                 27     7              13
 7 3RC                       5     2                 22     9              16
 8 3R                       86    34                 93    45             102
 9 4                         4     0                  5     0               8
10 X                        40    35                 75    31              36
11 Y                         1     6         

In [10]:
regions$Germline_Kc167[11, ] <- 0
regions$Somatic_Kc167[11, ] <- 0
regions$Kc167_S2[11, ] <- 0
print(regions)

# A tibble: 11 × 7
   region Germline_Somatic[,1]  [,2] Germline_Kc167[,1]  [,2] Germline_S2[,1]
   <chr>                 <int> <int>              <dbl> <dbl>           <int>
 1 2L                       59    42                 65    40              68
 2 2LC                       8     0                  7     4               6
 3 2RC                      20     1                 28    14              16
 4 2R                       42    31                 50    50              57
 5 3L                       54    43                 70    50              72
 6 3LC                      12     0                 27     7              13
 7 3RC                       5     2                 22     9              16
 8 3R                       86    34                 93    45             102
 9 4                         4     0                  5     0               8
10 X                        40    35                 75    31              36
11 Y                         1     6         

In [11]:
fisher.test(rbind(c(10, 100), c(0, 0)))


	Fisher's Exact Test for Count Data

data:  rbind(c(10, 100), c(0, 0))
p-value = 1
alternative hypothesis: true odds ratio is not equal to 1
95 percent confidence interval:
   0 Inf
sample estimates:
odds ratio 
         0 


In [16]:
contingency <- tibble(
  Table = colnames(regions)[-1],
  p = with_seed(
    0,
    regions %>%
      subset(select=-region) %>%
      sapply(
        \(table) fisher.test(
          table,
          simulate.p.value = TRUE,
          B = 1e+6
        )$p.value %>%
          round(digits = 6)
      )
  )
) %>%
  mutate(
    p.adjust = p.adjust(p),
    signif = structure(
      cut(p.adjust, c(-Inf, 1e-4, 1e-3, 1e-2, 5e-2, Inf)),
      levels = c("****", "***", "**", "*", ""),
      class = "factor"
    ),
    .keep = "unused",
  ) %>%
  print()

# A tibble: 6 × 3
  Table            p.adjust signif
  <chr>               <dbl> <fct> 
1 Germline_Somatic 0.000006 "****"
2 Germline_Kc167   0.0356   "*"   
3 Germline_S2      0.0832   ""    
4 Somatic_Kc167    0.000006 "****"
5 Somatic_S2       0.000036 "****"
6 Kc167_S2         0.000028 "****"


In [17]:
write.table(
  contingency,
  sep = "\t",
  quote = F,
  row.names = F
)

Table	p.adjust	signif
Germline_Somatic	6e-06	****
Germline_Kc167	0.035594	*
Germline_S2	0.083223	
Somatic_Kc167	6e-06	****
Somatic_S2	3.6e-05	****
Kc167_S2	2.8e-05	****


In [28]:
barpval <- regions %>%
  reframe(
    region,
    across(
      !region,
      \(Y) mapply(
        \(Y, i) fisher.test(
          rbind(
            Y[i,, drop=T],
            colSums(Y[-i,, drop=F])
          )
        )$p.value,
        list(Y),
        seq(nrow(Y))
      )
    )
  )
barcomparison <- regions["region"]
for (comparison in colnames(regions)[-1]) {
  comparison_names <- strsplit(comparison, "_")[[1]]
  name1 <- paste0(comparison_names[1], "_Earlier") %>%
    c(colnames(barcomparison), .) %>%
    make.unique() %>%
    tail(1)
  name2 <- paste0(comparison_names[2], "_Earlier") %>%
    c(colnames(barcomparison), .) %>%
    make.unique() %>%
    tail(1)
  barcomparison[name1] <- pull(regions, comparison)[, 1]
  barcomparison[name2] <- pull(regions, comparison)[, 2]
  barcomparison[paste0("Signif_", name1)] <- structure(
    cut(pull(barpval, comparison), c(-Inf, 1e-4, 1e-3, 1e-2, 5e-2, Inf)),
    levels = c("****", "***", "**", "*", ""),
    class = "factor"
  ) %>%
    replace(
      chisq.test(pull(regions, comparison))$residuals[, 1] < 0,
      ""
    )
  barcomparison[paste0("Signif_", name2)] <- structure(
    cut(pull(barpval, comparison), c(-Inf, 1e-4, 1e-3, 1e-2, 5e-2, Inf)),
    levels = c("****", "***", "**", "*", ""),
    class = "factor"
  ) %>%
    replace(
      chisq.test(pull(regions, comparison))$residuals[, 2] < 0,
      ""
    )
}
barcomparison

Warning message in chisq.test(pull(regions, comparison)):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(pull(regions, comparison)):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(pull(regions, comparison)):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(pull(regions, comparison)):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(pull(regions, comparison)):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(pull(regions, comparison)):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(pull(regions, comparison)):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(pull(regions, comparison)):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(pull(regions, comparison)):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(pull(regions, comparison)):
“Chi-squared ap

region,Germline_Earlier,Somatic_Earlier,Signif_Germline_Earlier,Signif_Somatic_Earlier,Germline_Earlier.1,Kc167_Earlier,Signif_Germline_Earlier.1,Signif_Kc167_Earlier,Germline_Earlier.2,⋯,Signif_Somatic_Earlier.1,Signif_Kc167_Earlier.1,Somatic_Earlier.2,S2_Earlier.1,Signif_Somatic_Earlier.2,Signif_S2_Earlier.1,Kc167_Earlier.2,S2_Earlier.2,Signif_Kc167_Earlier.2,Signif_S2_Earlier.2
<chr>,<int>,<int>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<int>,⋯,<fct>,<fct>,<int>,<int>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>
2L,59,42,,,65,40,,,68,⋯,,,45,49,,,54,64,,
2LC,8,0,*,,7,4,,,6,⋯,,*,1,2,,,5,1,,
2RC,20,1,***,,28,14,,,16,⋯,,**,2,16,,**,14,10,,
2R,42,31,,,50,50,,**,57,⋯,,**,46,41,,,80,51,**,
3L,54,43,,,70,50,,,72,⋯,,,56,56,,,69,69,,
3LC,12,0,**,,27,7,,,13,⋯,,***,0,13,,***,4,19,,**
3RC,5,2,,,22,9,,,16,⋯,,,6,3,,,3,4,,
3R,86,34,*,,93,45,,,102,⋯,,,54,84,,,87,73,,
4,4,0,,,5,0,,,8,⋯,,,6,0,**,,9,2,*,


In [29]:
write.table(
  barcomparison,
  sep = "\t",
  quote = F,
  row.names = F
)

region	Germline_Earlier	Somatic_Earlier	Signif_Germline_Earlier	Signif_Somatic_Earlier	Germline_Earlier.1	Kc167_Earlier	Signif_Germline_Earlier.1	Signif_Kc167_Earlier	Germline_Earlier.2	S2_Earlier	Signif_Germline_Earlier.2	Signif_S2_Earlier	Somatic_Earlier.1	Kc167_Earlier.1	Signif_Somatic_Earlier.1	Signif_Kc167_Earlier.1	Somatic_Earlier.2	S2_Earlier.1	Signif_Somatic_Earlier.2	Signif_S2_Earlier.1	Kc167_Earlier.2	S2_Earlier.2	Signif_Kc167_Earlier.2	Signif_S2_Earlier.2
2L	59	42			65	40			68	55			45	48			45	49			54	64		
2LC	8	0	*		7	4			6	3			0	6		*	1	2			5	1		
2RC	20	1	***		28	14			16	14			0	9		**	2	16		**	14	10		
2R	42	31			50	50		**	57	37			33	60		**	46	41			80	51	**	
3L	54	43			70	50			72	53			51	46			56	56			69	69		
3LC	12	0	**		27	7			13	12			0	12		***	0	13		***	4	19		**
3RC	5	2			22	9			16	9			12	5			6	3			3	4		
3R	86	34	*		93	45			102	63			51	64			54	84			87	73		
4	4	0			5	0			8	1			2	7			6	0	**		9	2	*	
X	40	35			75	31			36	44		*	82	40	****		43	38			38	78		***
Y	1	6		*	0	0			7	14		